# Network Analysis Project
## Crime Network Analysis
Professor: Michele Coscia

This notebook contains all of the code developed for the Network Analysis Course at ITU. We will be using...**finish**

**Add Description**

Group 10:  
Carl August Wismer ([cwis@itu.dk](mailto:cwis@itu.dk))  <br>
Crisanna Cornish ([ccor@itu.dk](mailto:ccor@itu.dk))  <br>
Danielle Dequin ([ddeq@itu.dk](mailto:ddeq@itu.dk))  <br>
Maria Do Carmo Madeira Santos Silva Passos de Sousa ([mdom@itu.dk](mailto:mdom@itu.dk))  <br>
Moneeca Abru Iftikhar Latif ([abml@itu.dk](mailto:abml@itu.dk))  <br>
Sabrina Fonseca Pereira ([sabf@itu.dk](mailto:sabf@itu.dk))  <br>

Created: 27-09-2021  
Last Modified: 01-11-2021

# Data Source

The data are a network of associations among suspects, victims, and/or witnesses involved in crimes in St. Louis in the 1990s. Data are derived from police records, via snowball sampling from five initial homicides. Left nodes are people, right nodes are crime events, and edges connect people to particular crimes events they were associated with. Metadata includes names, genders, and roles (suspects, victims, and/or witnesses).

This is an undirected, unweighted, bipartite network with 1380 nodes and 1476 edges.

Data can be downloaded [here](http://konect.cc/networks/moreno_crime/) or [here](https://networks.skewed.de/net/crime).

# Imports

In [ ]:
import numpy as np
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
from scipy.stats import linregress, pearsonr
from scipy.optimize import curve_fit
import powerlaw as pl

# PATHS

In [ ]:
DATA = '../Data/out.moreno_crime_crime'
ROLE = '../Data/rel.moreno_crime_crime.person.role'
NAME = '../Data/ent.moreno_crime_crime.person.name'
SEX =  '../Data/ent.moreno_crime_crime.person.sex'

# Constants

# Functions

In [ ]:
def fu(x, a, l):
    """A custom function which is a power law with its exponential truncation. From 'Atlas for the Aspiring
    Network Scientist' Chapter 6
    """
    return (x ** a) * np.exp(-l * x) 

def log_f(x, a, l):
    """The Logarithm of f, used to fit the log of the CCDF using curve_fit"""
    return np.log10(f(x, a, l))

# Load Data

In [ ]:
with open(DATA) as f:
    data = f.read().splitlines()
    
with open(ROLE) as f:
    role = f.read().splitlines()
    
with open(NAME) as f:
    name = f.read().splitlines()
    
with open(SEX) as f:
    sex = f.read().splitlines()
    
for i in range(len(data)):
    data[i] = 'p' + data[i] # adds 'p' to the People nodes to differentiate them

In [ ]:
B = nx.read_edgelist(data) # read edgelist in
print(len(B.nodes()), len(B.edges())) # sanity check (1380, 1476)

In [ ]:
graph = np.loadtxt(DATA, dtype=str)
a = np.unique(graph[:,0]) # people nodes
b = np.unique(graph[:,1]) # crime nodes
a = ['p' + a for a in a] # add string to differentiate

print(f"There are {len(a)} unique people and {len(b)} unique crime events.")

## Create Graph

In [ ]:
G = nx.Graph()

# add nodes
for j in range(len(a)):
    G.add_node(a[j], bipartite=0, data=True, name=name[j], sex=sex[j]) #Add attributes name and sex

G.add_nodes_from(b, bipartite=1, data=True)

# add edges
i = 0
for edge in B.edges():
    G.add_edge(edge[0], edge[1], role=role[i]) # why role is in a list?
    i+=1

In [ ]:
# sanity check
print(
    'Num. of nodes: {} \nNum. of edges: {} \nIs bipartite? {} \nIs connected? {}'.format(
        G.number_of_nodes(), 
        G.number_of_edges(), 
        nx.is_bipartite(G),
        nx.is_connected(G)
        )
    )

From the [official Networkx docs](https://networkx.org/documentation/stable/reference/algorithms/bipartite.html):

Many algorithms of the bipartite module of NetworkX require, as an argument, a container with all the nodes that belong to one set, in addition to the bipartite graph B. The functions in the bipartite package do not check that the node set is actually correct nor that the input graph is actually bipartite. If B is connected, you can find the two node sets using a two-coloring algorithm:

```py
>>> nx.is_connected(B)
True
>>> bottom_nodes, top_nodes = bipartite.sets(B)
```
However, if the input graph is not connected, there are more than one possible colorations. This is the reason why we require the user to pass a container with all nodes of one bipartite node set as an argument to most bipartite functions. In the face of ambiguity, we refuse the temptation to guess and raise an AmbiguousSolution Exception if the input graph for bipartite.sets is disconnected.

Using the bipartite node attribute, you can easily get the two node sets:

In [ ]:
top_nodes = {n for n, d in G.nodes(data=True) if d["bipartite"] == 0} # people
bottom_nodes = set(G) - top_nodes # crimes

## Adding metadata to nodes

In [ ]:
print(nx.get_node_attributes(G, 'name')['p1']) # check name of person 'p1' = 'AbelDennis'
print(nx.get_node_attributes(G, 'sex')['p1']) # check sex of person 'p1' = '1'

In [ ]:
nx.get_edge_attributes(G, 'role')['p1', '1']
# check role of edge between person p1 and crime 1 = 'Suspect'

# Finding Largest Connected Component

### *LCC:* Here we find that the largest connected component contains 1263 nodes (out of 1380 total nodes). This has a large majority of the nodes in the connected component. We do futher analysis to see if we should focus on this component alone as we continue.

In [ ]:
# Components in the graph sorted in descendent order
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)

# Selecting the biggest component
G0 = G.subgraph(Gcc[0])

In [ ]:
# Create lists of nodes for the LCC
top_nodes_lcc = {n for n, d in G0.nodes(data=True) if d["bipartite"] == 0} # people
bottom_nodes_lcc = set(G0) - top_nodes_lcc      # crimes

In [ ]:
# Number of nodes for each component and number of components
comp_list = []
for comp in Gcc:
    comp_len = len(comp)
    comp_list.append(comp_len)

BC_perc = round(comp_list[0]/sum(comp_list) * 100,3)

print(f'Number of nodes for each components: {comp_list} and number of components: {len(comp_list)}.')
print(f'The number of number of nodes in the biggest component represents {BC_perc} % of all nodes.')

# Exploratory Data Analysis (EDA)

## Average Clustering

### *Density:* probability that a random node pair is connected or, the number of edges in a network over the total possible number of edges that can exist given the number of nodes. 

In our case, we have a probablity of 0.003 which means that our network is quite sparse. Additionally, we see that the probability is very similar for the LCC.

In [ ]:
print(
    'Density: {} \nAvarage clustering: {}'.format(
        round(bipartite.density(G, bottom_nodes),3),
        round(bipartite.average_clustering(G),3)
        )
    )
print('\n')
print('Density LCC: {} \nAverage Clustering LCC: {}'.format(
        round(bipartite.density(G0, bottom_nodes_lcc),3), 
        round(bipartite.average_clustering(G0),3)
    )
)

## Another way to get Average Clustering

### *Clustering coefficient:* number of common neighbors around a node connected to each other. How often does a triad (set of three connected nodes) closes down into a triangle (three nodes that are all connected to each other)? 
#### Note_1: in the formula, we multiply by 3 because a single triangle closes three triads.
#### Note_2: local, global and average clustering coefficients are different.

In our case, almost half of the triads (0.43) in the network close to form a triangle. 


In [ ]:
cluster = bipartite.clustering(G)
print(f'Clustering of Entire Network: {round(sum(cluster.values())/len(cluster.values()),5)}')

cluster2 = bipartite.clustering(G0)
print(f'Clustering for LCC: {round(sum(cluster2.values())/len(cluster2.values()),5)}')

### **Looking for both density and clustering coefficient values, our network follows the real world network caracterists having few connections per node, but these connections tend to be clustered in the same neighborhood. Nodes tend to close triangles.**

## Global Clustering

In [ ]:
# This is the entire graph
global_clustering = bipartite.robins_alexander_clustering(G)
print(f'Global Clustering: {round(global_clustering,4)}')

# This is for only the LCC
global_clustering_lcc = bipartite.robins_alexander_clustering(G0)
print(f'Global Clustering LCC: {round(global_clustering_lcc, 4)}')

# Nodes stats

### *Betweenness centrality:* total number of paths that can pass through a node – excluding the ones for which it is the origin or the destination; how many paths would become longer if node v would disappear from the network?

The pattern of betweenness centrality between the LCC and the entire network is very similar.

In [ ]:
# For the Whole network
betweenness_centrality = bipartite.betweenness_centrality(G, top_nodes)

# For the LCC
betweenness_centrality_lcc = bipartite.betweenness_centrality(G0, top_nodes_lcc)

In [ ]:
betweenness_centrality['p815'], betweenness_centrality_lcc['p815']

In [ ]:
betweenness_centrality['p56'], betweenness_centrality_lcc['p56']

In [ ]:
max(betweenness_centrality.values()), max(betweenness_centrality_lcc.values())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,4))
ax1.set_title('Betweenness Centrality of Whole Network')
ax2.set_title('Betweenness Centrality of LCC')
ax1.hist(betweenness_centrality.values(), bins=10, log=True)
ax2.hist(betweenness_centrality_lcc.values(), bins=10, log=True)
plt.show();

### *Closeness centrality:* inverse average path length. The closer a node are on average to every other nodes, the more central it is.

In [ ]:
# For the Whole Network
closeness_centrality = bipartite.closeness_centrality(G, top_nodes)

# For the LCC
closeness_centrality_lcc = bipartite.closeness_centrality(G0, top_nodes_lcc)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,4))
ax1.set_title('Closeness Centrality of Whole Network')
ax2.set_title('Closeness Centrality of LCC')
ax1.hist(closeness_centrality.values(), bins=900)
ax1.set_xlim(.06,.19)
ax2.hist(closeness_centrality_lcc.values(), bins=75)
plt.show();

### *Avg degree:* average number of edges each node uses to connect to its neighbors. Depending on the number of nodes and the avg degree, we can have an idea of how connected or sparse our network is.

In [ ]:
# Node Degree
node_degree = bipartite.degrees(G, top_nodes)
#print(node_degree[1]) 
# node_degree[1] is showing the people nodes, and their corresponding degree
# node_degree[0] shows all the crime nodes, and their correponding degree

node_deg_dict = {}

for i in node_degree[1]:
    node_deg_dict[i[0]] = i[1] 
    

In [ ]:
node_deg_dict['p815']

In [ ]:
# People node with the most neighbors (Max left degree)
node_neighbors = {n: len(set(G.neighbors(n))) for n in top_nodes}
maxnode = max(node_neighbors, key = node_neighbors.get)
print(maxnode, node_neighbors[maxnode])

# People node in LCC with most neighbors
node_neighbors_lcc = {n: len(set(G0.neighbors(n))) for n in top_nodes_lcc}
maxnode_lcc = max(node_neighbors_lcc, key = node_neighbors_lcc.get)
print(maxnode_lcc, node_neighbors_lcc[maxnode_lcc])

In [ ]:
# Crime node with the most neighbors (Max right degree)
node_neighbors = {n: len(set(G.neighbors(n))) for n in bottom_nodes}
maxnode = max(node_neighbors, key = node_neighbors.get)
print(maxnode, node_neighbors[maxnode])

node_neighbors_lcc = {n: len(set(G0.neighbors(n))) for n in bottom_nodes_lcc}
maxnode_lcc = max(node_neighbors_lcc, key = node_neighbors_lcc.get)
print(maxnode_lcc, node_neighbors_lcc[maxnode_lcc])

# Degree Distribution

### *This is looking into the degree distribution of all nodes in the entire network and all nodes of the largest connected component to compare the entire network with the LCC. A loglog is applied to both to see if they follow a power law. The y axis shows the probability of a node having a degree equal to k.*

In [ ]:
# Degree Distribution of the Entire Network, Combining People and Crime Nodes
dd = Counter(dict(G.degree).values())
dd = pd.DataFrame(list(dd.items()), columns = ("Degree (k)", "p(k)")).sort_values(by = "Degree (k)")
dd["p(k)"] = dd["p(k)"]/(dd["p(k)"].sum())

# Degree Distribution of the LCC, Combining People and Crime Nodes
dd_lcc = Counter(dict(G0.degree).values())
dd_lcc = pd.DataFrame(list(dd_lcc.items()), columns = ("Degree (k)", "p(k)")).sort_values(by = "Degree (k)")
dd_lcc["p(k)"] = dd_lcc["p(k)"]/(dd_lcc["p(k)"].sum())

fig, axs = plt.subplots(2,2, figsize=(12,10)) 
dd.plot(kind = "scatter", x = "Degree (k)", y = "p(k)", ax=axs[0,0], color = "#e41a1c", title="Degree Distribution of All Nodes")
dd.plot(kind = "scatter", x = "Degree (k)", y = "p(k)", ax=axs[0,1], color = "#e41a1c", loglog = True, title="Loglog Degree Distribution of All Nodes")

dd_lcc.plot(kind = "scatter", x = "Degree (k)", y = "p(k)", ax=axs[1,0], color = "#e41a1c", title="Degree Distribution of LCC")
dd_lcc.plot(kind = "scatter", x = "Degree (k)", y = "p(k)", ax=axs[1,1], color = "#e41a1c", loglog = True, title="Loglog Degree Distribution of LCC")
#plt.savefig("../Report/degree_distributions.png")
plt.show()

## People Vs Crime Node Degree Distribution

### *This is looking into the degree distribution of the crime nodes and the people nodes separtely, to see if one of them stands out as having a very different distribution. This is broken down into the Entire Network, as well as the Largest-connected-component of the entire network. In all cases, it seems like the degree distribution is fat-tailed, with very few nodes having a high degree, and most nodes having a very low degree. Meaning most nodes are connected to very few other nodes. In other words, most people are involved in very few crimes.*

In [ ]:
def normalize(x):
    """Function to normalize the y count of the degrees, so that the y-values on the plots
    will be a probability of that degree, instead of the count of the degree."""
    total = sum(x.values())
    x = {
           k: v / total
           for k, v in x.items()
        }
    return x

In [ ]:
# This includes the entire graph
e_p = []
e_c = []
for k in G.nodes(): # Loop through all nodes
    if G.nodes[k]["bipartite"] == 0: # If People Nodes
        e_p.append(len(G.edges(k)))  # Append number of edges to each node
    else:                            # If Crime Nodes
        e_c.append(len(G.edges(k)))  # Append number of edges to each crime

e_cc = Counter(e_c)
e_cc= normalize(e_cc)

e_pp = Counter(e_p)
e_pp = normalize(e_pp)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(12,10))
fig.suptitle('Degree Distributions of the Network')

ax1.scatter(e_cc.keys(), e_cc.values())
ax1.set_title('Crime Nodes')
ax1.set_xlabel('Degree (k)')
ax1.set_ylabel('p(k)')

ax2.scatter(e_pp.keys(), e_pp.values())
ax2.set_title('People Nodes')
ax2.set_xlabel('Degree (k)')
ax2.set_ylabel('p(k)');

# This only deals with the LCC
# Lists of number of edges per node
e_p1 = [] # Number of edges connected to each People node
e_c1 = [] # Number of edges connected to each Crime node
for k in G0.nodes(): # loop through all nodes
    if G0.nodes[k]["bipartite"] == 0:  # If People Node
        e_p1.append(len(G0.edges(k)))  # Append number of edges to each node
    else:                              # If Crime Node
        e_c1.append(len(G0.edges(k)))  # Append number of edges to each node

e_cc1 = Counter(e_c1)
e_cc1 = normalize(e_cc1)

e_pp1 = Counter(e_p1)
e_pp1 = normalize(e_pp1)

#fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,4))
#fig.suptitle('Degree Distributions of the Largest Connected Component')

ax3.scatter(e_cc1.keys(), e_cc1.values())
ax3.set_title('Crime Nodes of LCC')
ax3.set_xlabel('Degree (k)')
ax3.set_ylabel('p(k)')

ax4.scatter(e_pp1.keys(), e_pp1.values())
ax4.set_title('People Nodes of LCC')
ax4.set_xlabel('Degree (k)')
ax4.set_ylabel('p(k)');

# CCDF
### *The Complement of a cumulative distribution function (CCDF) is the probability of finding a node of degree k or higher. It appears that, in log-log space, the relationship between degree and the number of nodes with a given degree is fixed.*

In [ ]:
# Cumulative Degree distribution
ccdf = dd.sort_values(by = "Degree (k)", ascending = False)
ccdf["cumsum"] = ccdf["p(k)"].cumsum()
ccdf["ccdf: P(X>=k)"] = ccdf["cumsum"] / ccdf["p(k)"].sum()
ccdf = ccdf[["Degree (k)", "ccdf: P(X>=k)"]].sort_values(by = "Degree (k)")

ccdf.plot(kind = "line", x = "Degree (k)", y = "ccdf: P(X>=k)", color = "#e41a1c", loglog = True)
#plt.savefig("../Report/degree_distribution_ccdf.png")
plt.show()

## Fit CCDF to PowerLaw
### *A power law is a functional relationship between two quantities, where a relative change in one quantity results in a proportional relative change in the other quantity.*

https://pythonhosted.org/powerlaw/

$p(k) \approx k^{-\alpha}$

$y = kx^{\alpha}$

Where: 
X and Y are variables of interest,
$\alpha$ is the law’s exponent,
$k$ is a constant.

In [ ]:
# Take the log of the ccdf
logcdf = np.log10(ccdf[["Degree (k)", "ccdf: P(X>=k)"]])
# find the slope, intercept, r-value, and p-value using linear regression
slope, log10intercept, r_value, p_value, std_err = linregress(logcdf["Degree (k)"], logcdf["ccdf: P(X>=k)"])

print("CCDF Fit: %1.4f x ^ %1.4f (R2 = %1.4f, p = %1.4f)" % (10 ** log10intercept, slope, r_value ** 2, p_value))
print("\n")

# fit the data to the power law
results = pl.Fit(ccdf["ccdf: P(X>=k)"])
# Select data above a specific degree
k_min = ccdf[ccdf["ccdf: P(X>=k)"] == results.power_law.xmin]["Degree (k)"]

print("Powerlaw CCDF Fit: %1.4f x ^ -%1.4f (k_min = %d)" % (10 ** results.power_law.Kappa, results.power_law.alpha, k_min))

In [ ]:
# Plot the best fit.
ccdf["fit"] = (10 ** results.power_law.Kappa) * (ccdf["Degree (k)"] ** -results.power_law.alpha)
ax = plt.gca()
ccdf.plot(kind = "line", x = "Degree (k)", y = "ccdf: P(X>=k)", color = "#e41a1c", loglog = True, ax = ax)
ccdf.plot(kind = "line", x = "Degree (k)", y = "fit", color = "#377eb8", loglog = True, ax = ax)
#plt.savefig("../Report/ccdf_fit.png")
plt.show()

## More Powerlaw Stuff
### *Explanation*... chapter 6 exercises

In [ ]:
# popt, pcov = curve_fit(log_f, ccdf["Degree (d)"], np.log10(ccdf["ccdf: P(X>=d)"]), p0 = (1, 1))
# ccdf["fit"] = ccdf.apply(lambda x: fu(x["Degree (d)"], popt[0], popt[1]), axis = 1)

# ax = plt.gca()
# ccdf.plot(kind = "line", x = "Degree (d)", y = "ccdf: P(X>=d)", color = "#e41a1c", loglog = True, ax = ax)
# ccdf.plot(kind = "line", x = "Degree (d)", y = "fit", color = "#377eb8", loglog = True, ax = ax)
# #plt.savefig("../Report/ccdf_fit.png")
# plt.show()

# Degree Distributions Per Node for Entire Network
### *Again, comparing degree distributions of the entire network, with people and crime nodes separated, but on the same axis.*

In [ ]:
# This includes the entire network

# Dictionary of Crime Nodes and Their Degree
crimes_degree = list(G.degree(b))
crimes_degree = pd.DataFrame(crimes_degree, columns = ("Node", "Degree")).sort_values(by = "Node")

# Dictionary of Crime Node Degrees and Their Frequencies
crimes_degree_count = Counter(dict(G.degree(b)).values())
crimes_degree_count = pd.DataFrame(list(crimes_degree_count.items()), columns = ("Degree (k)", "p(k)")).sort_values(by = "Degree (k)")
crimes_degree_count["p(k)"] = crimes_degree_count["p(k)"]/(crimes_degree_count["p(k)"].sum())


In [ ]:
# This includes the entire network

# Dictionary of People Nodes and Their Degree
ppl_degree = list(G.degree(a))
ppl_degree = pd.DataFrame(ppl_degree, columns = ("Node", "Degree")).sort_values(by = "Node")

# Dictionary of People Node Degrees and Their Frequencies
ppl_degree_count = Counter(dict(G.degree(a)).values())
ppl_degree_count = pd.DataFrame(list(ppl_degree_count.items()), columns = ("Degree (k)", "p(k)")).sort_values(by = "Degree (k)")
ppl_degree_count["p(k)"] = ppl_degree_count["p(k)"]/(ppl_degree_count["p(k)"].sum())

#ppl_degree_count

In [ ]:
# This includes the entire network
fig, ax = plt.subplots(figsize=(10,5))
ax = plt.gca()
crimes_degree_count.plot(kind = "scatter", x = "Degree (k)", y = "p(k)", color = "#fb00ff", ax = ax, alpha=0.9, s=200)
ppl_degree_count.plot(kind = "scatter", x = "Degree (k)", y = "p(k)", color = "#ff9d00", ax = ax, alpha=0.7, s=200)
ax.legend(["Crimes","People"])
plt.xticks(np.arange(0, 26, step=1))
plt.title("Degree Distributions per Node Category")
plt.show()

# Degree Distribution Using only the LCC
### *Now comparing the degree distributions of the people and crime nodes in the largest connected component on one axis. Here it follows the same pattern as that of the entire network, indicating that focusing on the largest connected component alone will maintain the integrity of the analysis on the entire network. In other words, the results on the LCC can be generalized towards the entire network, as it represents the entire network well.*

In [ ]:
# Create DataFrame of People Nodes and their Degree
p_deg_lcc = list(G0.degree(a))
p_deg_lcc = pd.DataFrame(p_deg_lcc, columns = ("Node", "Degree")).sort_values(by = "Node")

# DataFrame of People Node Degrees and their Frequency
p_deg_lcc_count = Counter(dict(G0.degree(a)).values())
p_deg_lcc_count = pd.DataFrame(list(p_deg_lcc_count.items()), columns = ("Degree (k)", "p(k)")).sort_values(by = "Degree (k)")
p_deg_lcc_count["p(k)"] = p_deg_lcc_count["p(k)"]/(p_deg_lcc_count["p(k)"].sum())

#p_deg_lcc_count

In [ ]:
# Create DataFrame of Crime Nodes and their Degree
c_deg_lcc = list(G0.degree(b))
c_deg_lcc = pd.DataFrame(c_deg_lcc, columns = ("Node", "Degree")).sort_values(by = "Node")

# Create DataFrame of Crime Node Degrees and their Frequency
c_deg_lcc_count = Counter(dict(G0.degree(b)).values())
c_deg_lcc_count = pd.DataFrame(list(c_deg_lcc_count.items()), columns = ("Degree (k)", "p(k)")).sort_values(by = "Degree (k)")
c_deg_lcc_count["p(k)"] = c_deg_lcc_count["p(k)"]/(c_deg_lcc_count["p(k)"].sum())

#crimes_degree_lcc_count

In [ ]:
# This Uses Only the LCC
fig, ax = plt.subplots(figsize=(8,5))
ax = plt.gca()
c_deg_lcc_count.plot(kind = "scatter", x = "Degree (k)", y = "p(k)", color = "#fb00ff", ax = ax, alpha=0.9, s=150)
p_deg_lcc_count.plot(kind = "scatter", x = "Degree (k)", y = "p(k)", color = "#ff9d00", ax = ax, alpha=0.7, s=150)

ax.legend(["Crimes","People"])
plt.xticks(np.arange(0, 26, step=1))
plt.title("Degree Distributions per Node Category Using Only LCC")
#plt.savefig("../Report/degree_distribution_lcc.png")
plt.show()

In [ ]:
# This Uses Only the LCC
fig, ax = plt.subplots(figsize=(8,5)) 
ax = plt.gca()
c_deg_lcc_count.plot(kind = "scatter", loglog=True, x = "Degree (k)", y = "p(k)", color = "#fb00ff", ax = ax, alpha=0.9, s=150)
p_deg_lcc_count.plot(kind = "scatter", loglog=True, x = "Degree (k)", y = "p(k)", color = "#ff9d00", ax = ax, alpha=0.7, s=150)

ax.legend(["Crimes","People"])
plt.title("Degree Distributions per Node Category Using Only LCC")
#plt.savefig("../Report/degree_distribution_loglog_lcc.png")
plt.show()

In [ ]:
# Degree distribution of Both Nodes in LCC
ccdf_lcc = dd_lcc.sort_values(by = "Degree (k)", ascending = False)
ccdf_lcc["cumsum"] = ccdf_lcc["p(k)"].cumsum()
ccdf_lcc["ccdf: P(X>=k)"] = ccdf_lcc["cumsum"] / ccdf_lcc["p(k)"].sum()
ccdf_lcc = ccdf_lcc[["Degree (k)", "ccdf: P(X>=k)"]].sort_values(by = "Degree (k)")

In [ ]:
# Crime Degree Distribution Using Only LCC
c_ccdf = c_deg_lcc_count.sort_values(by = "Degree (k)", ascending = False)
c_ccdf["cumsum"] = c_ccdf["p(k)"].cumsum()
c_ccdf["ccdf: P(X>=k)"] = c_ccdf["cumsum"] / c_ccdf["p(k)"].sum()
c_ccdf = c_ccdf[["Degree (k)", "ccdf: P(X>=k)"]].sort_values(by = "Degree (k)")

In [ ]:
# People Degree Distribution Using Only LCC
p_ccdf = p_deg_lcc_count.sort_values(by = "Degree (k)", ascending = False)
p_ccdf["cumsum"] = p_ccdf["p(k)"].cumsum()
p_ccdf["ccdf: P(X>=k)"] = p_ccdf["cumsum"] / p_ccdf["p(k)"].sum()
p_ccdf = p_ccdf[["Degree (k)", "ccdf: P(X>=k)"]].sort_values(by = "Degree (k)")

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12,4)) 
ax = plt.gca()

c_ccdf.plot(kind = "line", x = "Degree (k)", y = "ccdf: P(X>=k)", ax=axs[0], color = "#fb00ff", loglog = True)
p_ccdf.plot(kind = "line", x = "Degree (k)", y = "ccdf: P(X>=k)", ax=axs[0], color = "#ff9d00", loglog = True)
axs[0].legend(["Crimes","People"])
axs[0].set_title("CCDF per Node Category Using Only LCC")

ccdf_lcc.plot(kind = "line", x = "Degree (k)", y = "ccdf: P(X>=k)", ax=axs[1], color = "r", loglog = True)
axs[1].legend(["Both Nodes Combined"])
axs[1].set_title("CCDF For All Nodes in LCC")


plt.show()

# Fit CCDF for LCC to PowerLaw
## *Explanation*...

In [ ]:
logcdf_lcc = np.log10(ccdf_lcc[["Degree (k)", "ccdf: P(X>=k)"]])
slope, log10intercept, r_value, p_value, std_err = linregress(logcdf_lcc["Degree (k)"], logcdf_lcc["ccdf: P(X>=k)"])
print("CCDF Fit: %1.4f x ^ %1.4f (R2 = %1.4f, p = %1.4f)" % (10 ** log10intercept, slope, r_value ** 2, p_value))
print("\n")

results = pl.Fit(ccdf_lcc["ccdf: P(X>=k)"])
k_min = ccdf_lcc[ccdf_lcc["ccdf: P(X>=k)"] == results.power_law.xmin]["Degree (k)"]
print("Powerlaw CCDF Fit: %1.4f x ^ -%1.4f (k_min = %d)" % (10 ** results.power_law.Kappa, results.power_law.alpha, k_min))

In [ ]:
# Plot the best fit.
ccdf_lcc["fit"] = (10 ** results.power_law.Kappa) * (ccdf_lcc["Degree (k)"] ** -results.power_law.alpha)
ax = plt.gca()
ccdf_lcc.plot(kind = "line", x = "Degree (k)", y = "ccdf: P(X>=k)", color = "#e41a1c", loglog = True, ax = ax)
ccdf_lcc.plot(kind = "line", x = "Degree (k)", y = "fit", color = "#377eb8", loglog = True, ax = ax)
#plt.savefig("../Report/ccdf_lcc_fit.png")
plt.show()

# Projected Plots of the Entire Network

In [ ]:
# The Entire Network Projected onto People
P = bipartite.weighted_projected_graph(G, top_nodes)
plt.rcParams.update({'figure.figsize': (20, 10)})
pos = nx.spring_layout(P, seed=42)
nx.draw_networkx(P, pos, node_size=20, with_labels=False)

In [ ]:
# The Entire Network projected onto Crimes
C = bipartite.weighted_projected_graph(G, bottom_nodes)
plt.rcParams.update({'figure.figsize': (20, 10)})
pos = nx.spring_layout(C, seed=42)
nx.draw_networkx(C, pos, node_size=20, with_labels=False)
#nx.draw(C, node_size=20)

# Plots of Largest Connected Component

In [ ]:
# plot only biggest component
plt.rcParams.update({'figure.figsize': (15, 10)})
pos = nx.spring_layout(G0)
posB = nx.bipartite_layout(G0, top_nodes)
biggest_comp_graph = nx.draw(G0, pos, node_size=20)

## Projected Graphs From Only the LCC

In [ ]:
# Just the People from the LCC
P = bipartite.weighted_projected_graph(G0, top_nodes_lcc)
plt.rcParams.update({'figure.figsize': (15, 10)})
pos = nx.spring_layout(P, seed=42)
nx.draw_networkx(P, pos, node_size=20, with_labels=False)

#nx.draw(P, node_size=10)

In [ ]:
#P.degree()

In [ ]:
# node = G0.nodes['p271']
x = list(G0.neighbors('p715'))
for i in x:
    print(i, list(G0.neighbors(i)))

In [ ]:
from collections import defaultdict
con = defaultdict(int)
for a,b in P.edges:
    con[a] += 1
    con[b] += 1
print(sorted(con.values(), reverse = True))

In [ ]:
# Only Crimes from the LCC
C = bipartite.weighted_projected_graph(G0, bottom_nodes_lcc)
plt.rcParams.update({'figure.figsize': (15, 10)})
pos = nx.spring_layout(C, seed=42)
nx.draw_networkx(C, pos, node_size=20, with_labels=False)

#nx.draw(C, node_size=10)

In [ ]:
# Biggest component bipartite plot
nx.draw_networkx(G0, pos = nx.drawing.layout.bipartite_layout(G0, top_nodes), 
                 width = 0.1, linewidths = 0.5, node_size = 0.5, with_labels = False)

## Non-square adjacency matrix, and stocastic matrix

## Add explanation...

In [ ]:
# Non-square adjacency matrix
adjmat = nx.algorithms.bipartite.matrix.biadjacency_matrix(G, top_nodes)
print(adjmat.shape)

In [ ]:
# Project along smaller axis
if adjmat.shape[0] == 551:
    adjmat_proj = adjmat.dot(adjmat.T)
else:
    adjmat_proj = adjmat.T.dot(adjmat)

In [ ]:
# Make stocastic
adjmat_proj_stoc = adjmat_proj / adjmat_proj.sum(axis = 1)
print(adjmat_proj_stoc.shape)

# Projections of suspect graph
 Projections of suspect graph  
 From the BCC extracted only the suspect edges and created a new graph that is not connected but still bipartite. From there projected onto people nodes.
- [ ] Figure out how to keep crime name as edge attribute

## Extract suspect edges
Created a new graph with only suspects to do further analysis (try projections).

In [ ]:
# selected just the suspect edges
suspect_edges = [(p,c) for p,c,e in G0.edges(data=True) if e['role'] == 'Victim']
#print(suspect_edges)

In [ ]:
# getting all nodes that have suspect edges
sus_nodes = pd.DataFrame(suspect_edges, columns=['node', 'crime']).iloc[:,0].unique()
sus_nodes = set(sus_nodes)

## Suspect subgraph

In [ ]:
GS = G0.edge_subgraph(suspect_edges)

In [ ]:
print(
    'Num. of nodes: {} \nNum. of edges: {} \nIs bipartite? {} \nIs connected? {}'.format(
        GS.number_of_nodes(), 
        GS.number_of_edges(), 
        nx.is_bipartite(GS),
        nx.is_connected(GS)
        )
    )

In [ ]:
pos = nx.spring_layout(GS, seed=42)
nx.draw(GS, pos, node_size=10)

## Projection on to suspect nodes (people)

In [ ]:
nx.is_bipartite(GS)

In [ ]:
# get the people and crime nodes
people_nodes_GS = {n for n, d in GS.nodes(data=True) if d["bipartite"] == 0}
crime_nodes_GS = set(GS) - people_nodes_GS

In [ ]:
# projection of suspect graph
weighted_suspect_graph = nx.bipartite.weighted_projected_graph(GS, people_nodes_GS, ratio=False)
list(weighted_suspect_graph.edges(data=True))
# weight is the number of shared neighbors 

In [ ]:
# give the number of shared neighbors
weights = list(nx.get_edge_attributes(weighted_suspect_graph, 'weight').values())

# plot weights
labels, counts = np.unique(weights, return_counts=True)
plt.bar(labels, counts, align='center', log=True)
plt.gca().set_xticks(labels)
plt.title("Number of Shared Neighborns on Suspect Projection Network")
plt.ylabel("Frequency")
plt.xlabel("Number of Shared Neighbors")

#plt.hist(weights, bins = 10, log=True)
plt.show()

In [ ]:
nx.draw_spring(weighted_suspect_graph,node_size=10)

In [ ]:
from collections import defaultdict
con = defaultdict(int)
for a,b in weighted_suspect_graph.edges:
    con[a] += 1
    con[b] += 1
print(sorted(con.values(), reverse = True))

In [ ]:
print(
    'Num. of nodes: {} \nNum. of edges: {} \nIs bipartite? {} \nIs connected? {}'.format(
        weighted_suspect_graph.number_of_nodes(), 
        weighted_suspect_graph.number_of_edges(), 
        nx.is_bipartite(weighted_suspect_graph),
        nx.is_connected(weighted_suspect_graph)
        )
    )

In [ ]:
nx.number_connected_components(weighted_suspect_graph)

In [ ]:
# Components in the graph sorted in descendent order
sort_sus = sorted(nx.connected_components(weighted_suspect_graph), key=len, reverse=True)

# Selecting the biggest component
GS2 = weighted_suspect_graph.subgraph(sort_sus[0])

In [ ]:
ew = nx.get_edge_attributes(GS2, 'weight')

In [ ]:
options = {
    'node_color': 'orange',
    'node_size': 30,
    'edge_color': 'gray', 
    # 'linewidths': 1, 
    'width': list(ew.values()),
    'font_size': 8,
    'with_labels': False
}

In [ ]:
l = nx.shell_layout(GS2)
nx.draw(GS2, l, **options)

In [ ]:
nx.draw_kamada_kawai(GS2, **options)

# Find LCC from Projection

In [ ]:
# Components in the graph sorted in descendent order
P_Gcc = sorted(nx.connected_components(GS), key=len, reverse=True)

# Selecting the biggest component
P_G0 = GS.subgraph(P_Gcc[0])

In [ ]:
# Number of nodes for each component and number of components
comp_list = []
for comp in P_Gcc:
    comp_len = len(comp)
    comp_list.append(comp_len)

print(f'Number of nodes for each component:\n{comp_list} \n\nNumber of components: {len(comp_list)}')

# Adjacency Matrix

Central Place Theory.
Chapter 29 in the book.

The Adjacency matrix of our Projected LCC looks like a discrete model. Which imposes a strict core-periphery structure. This also does not seem to show any smaller communities extending off of the core.

In [ ]:
# Get the degrees of the nodes
degree = dict(P_G0.degree)

# Figure out which node is of which type
rows, cols = nx.algorithms.bipartite.sets(P_G0)

# Sort rows and columns according to their degree
rows = {n: degree[n] for n in rows}
cols = {n: degree[n] for n in cols}
rows = [n[0] for n in sorted(rows.items(), key = lambda x : x[1], reverse = True)]
cols = [n[0] for n in sorted(cols.items(), key = lambda x : x[1], reverse = True)]

In [ ]:
# Get the biadjacency matrix with the properly sorted rows and columns
B = nx.algorithms.bipartite.matrix.biadjacency_matrix(P_G0, rows, column_order = cols)
plt.figure(figsize=(4,8))
plt.spy(B, markersize = 3)
plt.show()

# Finding Which Nodes Connect to Which Cores

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.core.core_number.html#networkx.algorithms.core.core_number

In [ ]:
# This finds the cores of the entire network, and counts how many nodes are connected to each core
c_whole = nx.algorithms.core.core_number(G)
c_whole_count = Counter(c_whole.values())
c_whole_count

In [ ]:
# This finds the cores of the projected network of suspects
c = nx.algorithms.core.core_number(P_G0)
count_c = Counter(c.values())
print(count_c)
print(f"Core 1 has {count_c[1]} Nodes attached \nCore 2 has {count_c[2]} Nodes Attached")

# Degree Assortativity
Draw the degree assortativity plots of this network using first an **edge-centric strategy**.<br>
Based on Exercise 27.1<br>
https://www.networkatlas.eu/exercise.htm?c=27&e=1

In [ ]:
# Get the degrees of the nodes from the LCC
degrees = dict(G0.degree)

# Edge-centric plot
edge_assort = [(degrees[e[0]], degrees[e[1]]) for e in G0.edges]
edge_assort = pd.DataFrame(data = edge_assort, columns = ("k1", "k2"))
edge_assort = edge_assort.groupby(by = ["k1", "k2"]).size().reset_index().rename(columns = {0: "count"}).sort_values(by = "count")

# Taking the log count of edges with nodes with the same degrees
edge_assort["count"] = np.log(edge_assort["count"])

In [ ]:
# Plotting the edge-centric plot for the LCC
ax = plt.gca()
plt.scatter(edge_assort["k1"], edge_assort["k2"], c = edge_assort["count"], cmap = "Reds")
ax.set_yscale("log")
ax.set_xscale("log")
plt.colorbar(ax=ax,label="# Of Edges")
plt.xlabel("Node Degree")
plt.ylabel("Node Degree")
plt.show()

In [ ]:
# Now do the same for the LCC of the Projected Suspect Network to compare
# Get the degrees of the nodes from the LCC
degrees2 = dict(P_G0.degree)

# Edge-centric plot
edge_assort = [(degrees2[e[0]], degrees2[e[1]]) for e in P_G0.edges]
edge_assort = pd.DataFrame(data = edge_assort, columns = ("k1", "k2"))
edge_assort = edge_assort.groupby(by = ["k1", "k2"]).size().reset_index().rename(columns = {0: "count"}).sort_values(by = "count")

# Taking the log count of edges with nodes with the same degrees
edge_assort["count"] = np.log(edge_assort["count"])

In [ ]:
# Plotting the edge-centric plot for the LCC of the Projected Network

ax = plt.gca()
plt.scatter(edge_assort["k1"], edge_assort["k2"], c = edge_assort["count"], cmap = "Reds")
#ax.set_yscale("log")
#ax.set_xscale("log")
plt.colorbar(ax=ax, label="# Of Edges")
plt.xlabel("Node Degree")
plt.ylabel("Node Degree")
plt.show()

### *This plot does not seem to visualize a high possibility of degree assortivity. (As that would be visualized with perhaps a lot of points plotted on the off diagonal, whereas here there are barely any).*

# Degree Assortativity
Using a node-centric approach. We plot each node and compare a node’s degree with the average degree of its neighbors. In a **degree assortative** network, we expect to see a positive correlation: the more connections the node has, the more connections, on average, its neighbors have.

This also means that you should perform a power fit. The exponent of such a fit tells you whether the network is degree assortative (if it’s positive), disassortative (if it’s negative). 

It seems the LCC is disassortative: users with few friends likely attach to hubs.

In [ ]:
# Two vectors with the degree of a node in one, and the average neighbor degree in the other
node_assort = [(degrees[n], degrees[n2]) for n in G0.nodes for n2 in G0.neighbors(n)]
node_assort = pd.DataFrame(data = node_assort, columns = ("k", "neigh_k"))
node_assort = node_assort.groupby(by = "k").mean().reset_index()

In [ ]:
# Plotting the node-centric plot
ax = plt.gca()
plt.scatter(node_assort["k"], node_assort["neigh_k"])
#ax.set_yscale("log")
ax.set_xscale("log")
plt.show()

### Degree Assortativity Coefficient

In [ ]:
# Two vectors with the degrees at each endpoints of edges
x1 = []
y1 = []
for e in G0.edges:
    x1.append(degrees[e[0]])
    y1.append(degrees[e[1]])
    x1.append(degrees[e[1]])
    y1.append(degrees[e[0]])

In [ ]:
# Edge-centric assortativity is the pearson correlation of such vectors
r, p = pearsonr(x1, y1)
print("Degree assortativity coefficient: %1.4f (p-value < 0.001? %s)" % (r, p < 0.001))

In [ ]:
# Node-centric assortativity is the power-fit of such vectors
slope, intercept, r_value, p_value, std_err = linregress(np.log(node_assort["k"]), np.log(node_assort["neigh_k"]))
print("Degree assortativity coefficient: %1.4f (p-value < 0.001? %s)" % (slope, p_value < 0.001))

## Check LCC

In [ ]:
# First checking with the LCC of the Entire Network
# Get the degrees of the nodes from the LCC
degrees3 = dict(G0.degree)

# Let's make a new figure
plt.clf()

# Node-centric plot
node_assort = [(degrees3[n], degrees3[n2]) for n in G0.nodes for n2 in G0.neighbors(n)]
node_assort = pd.DataFrame(data = node_assort, columns = ("k", "neigh_k"))
node_assort = node_assort.groupby(by = "k").mean().reset_index()

In [ ]:
# Plotting the node-centric plot
ax = plt.gca()
plt.scatter(node_assort["k"], node_assort["neigh_k"])
#ax.set_yscale("log")
ax.set_xscale("log")
plt.xlabel("Node Degree")
plt.ylabel("Average Degree of a Node's Neighbor's")
plt.show()

Compute degree assortativity of graph.

Assortativity measures the similarity of connections in the graph with respect to the node degree.

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.assortativity.degree_pearson_correlation_coefficient.html

In [ ]:
r = nx.degree_pearson_correlation_coefficient(G0)
print(f"{r:3.1f}")

## Check LCC of Projected Suspect Network

In [ ]:
# Now checking with the LCC of the Projected Suspect Network
# Get the degrees of the nodes from the P_LCC
degrees = dict(P_G0.degree)

# Let's make a new figure
plt.clf()

# Node-centric plot
node_assort = [(degrees[n], degrees[n2]) for n in P_G0.nodes for n2 in P_G0.neighbors(n)]
node_assort = pd.DataFrame(data = node_assort, columns = ("k", "neigh_k"))
node_assort = node_assort.groupby(by = "k").mean().reset_index()

In [ ]:
# Plotting the node-centric plot
ax = plt.gca()
plt.scatter(node_assort["k"], node_assort["neigh_k"])
#ax.set_yscale("log")
ax.set_xscale("log")
plt.xlabel("Node Degree")
plt.ylabel("Average Degree of a Node's Neighbor's")
plt.show()

In [ ]:
r = nx.degree_pearson_correlation_coefficient(P_G0)
print(f"{r:3.1f}")

# Checking for Assortativity in Categorical Attributes (Homophily)

### *Assortativity measures the similarity of connections in the graph with respect to the given attribute. Perfect assortativity is 1, while perfect disassortativity is -1. The whole network is quite disassortative.*

In [ ]:
# Assortativity coefficient of the Entire Network
nx.attribute_assortativity_coefficient(G, "sex")

In [ ]:
# Assortativity coefficient of the LCC
nx.attribute_assortativity_coefficient(G0, "sex")

# Configuration Model

### This is used to compare what we find. Is what we find special for our network? Or is it something that would happen randomly?

In [ ]:
# To compare to a null model based on the LCC, unipartite.  This is not complete... it should compare the degrees of the 
# Projected people network
jimmyJo = list(dict(G0.degree).values())
CM = nx.configuration_model(jimmyJo)
nx.draw(CM, node_size=10)

In [ ]:
# A bipartite configuration model based on the entire dataset
Bi_CM = nx.algorithms.bipartite.generators.configuration_model(e_p, e_c)
#nx.draw(Bi_CM, node_size=10)

In [ ]:
# A bipartite that is only the LCC
Bi_CM_0 = nx.algorithms.bipartite.generators.configuration_model(e_p1, e_c1)
nx.draw(Bi_CM_0, node_size=10)